# Pull ESPN Ownership Data

### MSP 9.4.2017


In [41]:
from IPython.display import HTML
HTML('''<script>code_show=true; function code_toggle() { if (code_show){ $('div.input').hide();} else { $('div.input').show(); } code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
# boilerplate imports

import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import matplotlib as mpl
cmap = cm.gnuplot




In [2]:
# imports for scraping

from bs4 import BeautifulSoup

import requests

import operator



### Change In Ownership

Also set up an archive.

In [3]:

Ownership = {}


for tablenum in range(0,13):

    # cycle through all positions
    r  = requests.get('http://games.espn.com/flb/addeddropped?slotCategoryId='+str(tablenum))

    data = r.text

    soup = BeautifulSoup(data)

    tables = soup.findAll('table')


    for table in tables: 

        headings = [th.get_text() for th in table.find("tr").find_all("th")]
        #print headings


        for row in table.find_all("tr")[1:]: 

        #    sav = [td.find('a') for td in row.find_all("td")]
            sav = [td.get_text() for td in row.find_all("td")]

            w = sav.index(u'\xa0')

            #print (sav[1].split(',')[0]).strip('*')

            try:
                # block out all non-rankings
                xx = float(sav[0])

                #print sav[0:w]
                #print sav[w+1:len(sav)]

                # added
                Ownership[(sav[1].split(',')[0]).strip('*')] = float(sav[5])

                #dropped
                Ownership[(sav[w+2].split(',')[0]).strip('*')] = float(sav[w+6])

            except:
                pass

            #[u'RK', u'PLAYER, TEAM', u'POS', u'LAST', u'CUR', u'7-DAY +/-']
        

today = datetime.date.today()

f = open('ownership-archive/'+str(today)+'.txt','w')

for index,key in enumerate(Ownership.keys()):
    print >>f,key,Ownership[key]
    
f.close()

In [7]:
print l[0]

positives = ['congratulates']

print np.sum(np.array([(x in l[0]) for x in positives]))



The Houston Astros scuffled through a forgettable mid-season stretch, but have rediscovered their form with the addition of Justin Verlander to ...
0


In [8]:
negatives = ['rehab','injure','disable','DL','suspension','surgery','injury']

#[(x in string_in) for x in negatives]

def check_words(string_in,words):
    total = [(x in string_in) for x in words]
    return np.sum(np.array(total))
    

for index,key in enumerate(Ownership.keys()):
    if Ownership[key] < -5.:
        print key
        print '--------------'
        l = scrape_news_summaries(key)
        print l[0]
        print check_words(l[0],negatives),' ',
        print check_words(l[1],negatives),' ',
        print check_words(l[2],negatives)
        print ''



Corey Dickerson
--------------
Corey Dickerson went 0-for-3 at the plate with a strikeout as the Rays lost the Cubs on ... Dickerson has been slumping at the plate for the Rays.
0   0   0

Welington Castillo
--------------
Castillo is not in the lineup for Tuesday's game against the Red Sox. It's surprising Castillo is sitting out Tuesday against left-hander Drew ...
0   0   0

Starling Marte
--------------
Pirates' Starling Marte: Not in lineup Monday. by RotoWire Staff; 12h ago • 1 min read Update 12h ago. Marte (finger) is not in the lineup for Monday's game ...
0   0   0

Eduardo Nunez
--------------
#36 Eduardo Nunez grimaces as he run to first in the second inning of the Red Sox vs. Tampa Bay MLB game at Fenway Park. Saturday ...
0   0   0

Jedd Gyorko
--------------
It sounds like Gyorko will be cleared to run the bases and play the field Tuesday after being limited to pinch-hitting duty in his first several days ...
0   0   0

Jorge Polanco
--------------
MINNEAPOLIS – Twins sh

In [9]:
for index,key in enumerate(Ownership.keys()):
    if Ownership[key] > 5.:
        print key
        print '--------------'
        l = scrape_news_summaries(key)
        print l[0]
        print ''




Matt Olson
--------------
Matt Olson did it again on Monday. In the 3rd inning he launched yet another home run, this time against the Tigers in his first trip to Comerica ...

Christian Vazquez
--------------
This interview was conducted in Spanish and has been translated. Read it in Spanish here. When Christian Vazquez was promoted to the big ...

Mike Zunino
--------------
That's right. For what feels like the first time since he made his major league debut, Michael Accorsi Zunino is not at the top of Mariners' fans' list ...

Tommy Pham
--------------
Tommy Pham currently ranks 10th in the National League in fWAR. Of course, Mr. Pham's season did not begin until May 5. For the first month ...

Mitch Haniger
--------------
Seattle Mariners' Mitch Haniger watches a solo home run off Los Angeles Angels starting pitcher Andrew Heaney during the first inning.

Yoan Moncada
--------------
HOUSTON — Another two solid weeks to finish the season would take second baseman Yoan Moncada into t

In [51]:
print Ownership['Justin Verlander']

KeyError: 'Justin Verlander'

### News Alerts

Another interesting thing is grabbing Google News findings on players to see if we can determine what the narrative leading to the explosion in pickups or drops is coming from.

Good keywords:
1. Trade
2. Hot
3. Streak
4. Fire
5. Homer


Bad keywords:
1. Cold
2. Bench
3. Disabled/DL
4. Injure
5. Suspension



In [6]:
#https://stackoverflow.com/questions/39354587/scraping-google-news-with-beautifulsoup-returns-empty-results

from bs4 import BeautifulSoup
import requests
import time
from random import randint


def scrape_news_summaries(s):
    time.sleep(randint(0, 2))  # relax and don't let google be angry
    
    r = requests.get("http://www.google.com/search?q="+s+"&tbm=nws")    
    # Print the status code
    #print(r.status_code)  
    
    content = r.text
    #print content
    
    news_summaries = []
    soup = BeautifulSoup(content, "html.parser")
    #st_divs = soup.find_all("div", class_="gs-info")
    st_divs = soup.findAll("div", {"class": "st"})
    #st_divs = soup.findAll("div", {"class": "rc"})


    for st_div in st_divs:
        #print st_div
        news_summaries.append(st_div.text)
    return news_summaries




#http://www.espn.com/search/results?q=Alex+Wood
#r = requests.get("http://www.google.com/search?q="+s+"&tbm=nws")

l = scrape_news_summaries("Justin+Verlander")

for n in l:
    print(n)
    print('')

The Houston Astros scuffled through a forgettable mid-season stretch, but have rediscovered their form with the addition of Justin Verlander to ...

31, the Astros acquired Justin Verlander. He has made three starts with Houston and won them all, allowing two runs and just 10 hits over 21 ...

"On behalf of the Detroit Tigers, we sincerely thank Justin Verlander for his remarkable 13 seasons of dedication to the organization," Tigers ...

What would Jeff Luhnow, general manager of the Houston Astros, be feeling if he failed to pull the trigger on a Justin Verlander deal? Regret?

The lure of pitching for a contender swayed Justin Verlander to accept a trade from the Tigers to the Astros. He makes his first home start with ...

ANAHEIM -- Astros pitcher Justin Verlander is scheduled to make his Minute Maid Park debut on Sunday against the Mariners. Astros manager A.J. Hinch laid ...

Justin Verlander has never won a game in the World Series. He may ... Verlander is one of the best pitch

### Stats Alerts

We should also scrape the fangraphs gamelog page to see if there is anything obvious there.

### Crazy

It would be awesome to try and make a model to predict ownership data based on news stories.

In [63]:
r  = requests.get('http://www.espn.com/mlb/player/_/id/33095/jose-abreu')

data = r.text

soup = BeautifulSoup(data)

fantasy_table = soup.find_all('div', id='fantasy-content')

print fantasy_table#.find("tr")

#for table in tables: 

#        headings = [th.get_text() for th in table.find("tr").find_all("th")]
#        #print headings

#<div class="mod-content" id="fantasy-content">


#(row in table.find_all("tr", class_="grid_postseason"))

[<div class="mod-content" id="fantasy-content">{"playerId":5600,"percentChange":-0.2,"averageDraftPosition":46.9,"percentOwned":96.8,"playerRaterSEASON":7.2,"playerRater7DAY":2.3,"mostRecentNews":{"news":"Abreu went 1-for-4 with a two-run home run during Sunday's win over Tampa Bay.","spin":"Abreu missed the previous three games with an elbow injury, but Sunday's blast affirmed his health. With a .301/.354/.540 slash line, 27 homers, 82 RBI and 72 runs for the campaign, the 30-year-old first baseman is in line to post another rock-solid showing. The supporting cast surrounding him in the White Sox's lineup limits his upside, but the slugger continues to move the fantasy needle nonetheless.","date":"Sun Sep 3"},"fullName":"Jose Abreu","seasonOutlook":{"outlook":"Though in many respects, Abreu's performance has regressed since he broke into the majors in 2014, let's not understate the historical significance of his first three big-league seasons: He, along with Joe DiMaggio and Albert Pu